
## Dataset
The used dataset is the [Sentiment polarity datasets v2.0](http://www.cs.cornell.edu/people/pabo/movie-review-data/). The dataset contains 1000 positive and 1000 negative processed reviews that is stored in the `review_polarity` folder.

- Inside the `review_polarity` folder, `txt_sentoken` contains 2000 processed down-cased text files, seperated into to category sub-folder, namely `neg` (i.e. negative) and `pos` (i.e. positive). Each sub-folder contains 1000 review text files in that category. 

- `poldata.README.2.0` stores all the meta information of this dataset.

## Import Libraries

In [104]:
import numpy as np

from sklearn.datasets import load_files  

# nltk
from nltk import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from itertools import chain
from nltk.stem import WordNetLemmatizer

## 1. Import Data

Import the data using [**load_files**](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html).
The loaded `movie_data` is a dictionary, with the following attributes:
* `data` - a list of text reviews
* `target` - the corresponding label of the text reviews (integer index)
* `target_names` - the names of target classes.
* `filenames` - the filenames holding the dataset.

In [6]:
movie_data = load_files(r"review_polarity/txt_sentoken")  

In [12]:
movie_data['filenames']

array(['review_polarity/txt_sentoken/neg/cv677_18938.txt',
       'review_polarity/txt_sentoken/neg/cv980_11851.txt',
       'review_polarity/txt_sentoken/pos/cv239_3385.txt', ...,
       'review_polarity/txt_sentoken/pos/cv652_13972.txt',
       'review_polarity/txt_sentoken/neg/cv559_0057.txt',
       'review_polarity/txt_sentoken/neg/cv684_12727.txt'], dtype='<U48')

In [14]:
movie_data['target']

array([0, 0, 1, ..., 1, 0, 0])

In [16]:
movie_data['target_names']

['neg', 'pos']

In [30]:
# test whether it matches
emp = 10 # an example
movie_data['filenames'][emp], movie_data['target'][emp] # from the file path we know that it's the correct class

('review_polarity/txt_sentoken/pos/cv267_14952.txt', 1)

#### Define reviews and sentiments

In [26]:
reviews = movie_data.data

sentiments= movie_data.target  

In [28]:
reviews[emp]

b'while it was part of former yugoslavia , my country , croatia , used to be almost completely ignored by western films in general . \nwords " croatia " and " croatians " were unheard , even in movies whose plots actually took place on croatian locations . \nthe rare exception was the dogs of war , 1980 adaptation of frederick forsyth\'s bestselling novel , yet that rare occasion only added insult to the injury , by mentioning croatians in most unflattering context . \nin a single line they are referred as a bunch of mean people , probably terrorists , who happen to remove parts of arm dealers\' stomachs when they are unsatisfied with the quality of merchandise . \nsame line also illustrates the milieu where the movie takes place - dark and violent world of shady international deals with the blurred boundaries between business , politics and crime . \nthe movie protagonist , shannon ( christopher walken ) is a mercenary who spent the best years of his life fighting numerous little wars

In [32]:
sentiments[emp]

1

## 2. Text Preprocessing

### 2.1. Tokenization

> Each review text is read as a byte object (see there is a `b` in front of each review text if you print it out), however, the tokenizer cannot apply a string pattern on a bytes-like object. To resolve this, each review text is decoded with 'utf-8'


#### Utilities

In [40]:
def tokenizeReview(raw_review):
    """
        This function first convert all words to lowercases, 
        it then segment the raw review into sentences and tokenize each sentences 
        and convert the review to a list of tokens.
    """        
    review = raw_review.decode('utf-8') # convert the bytes-like object to python string, need this before we apply any pattern search on it
    nl_review = review.lower() # cover all words to lowercase
    
    # segament into sentences
    sentences = sent_tokenize(review)
    
    # tokenize each sentence
    pattern = r'''(?x)          # set flag to allow verbose regexps
        (?:[A-Za-z]\.)+        # abbreviations, e.g. U.S.A.
      #| \w*[\$£]?(?:\d+(?:,\d+)?)+(?:\.\d+)?%?\w*  # numbers, currency and percentages, e.g. $12.40, 82%
      | [A-Za-z]+(?:[-'][A-Za-z]*)?        # words with optional internal hyphens and apostrophes
    '''
    tokenizer = RegexpTokenizer(pattern) 
    token_lists = [tokenizer.tokenize(sen) for sen in sentences]
    
    # merge them into a list of tokens
    tokenised_review = list(chain.from_iterable(token_lists))
    return tokenised_review

In [61]:

def stats_print(tk_reviews):
    words = list(chain.from_iterable(tk_reviews)) # we put all the tokens in the corpus in a single list
    vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words
    lexical_diversity = len(vocab)/len(words)
    print("Vocabulary size: ",len(vocab))
    print("Total number of tokens: ", len(words))
    print("Lexical diversity: ", lexical_diversity)
    print("Total number of reviews:", len(tk_reviews))
    lens = [len(article) for article in tk_reviews]
    print("Average review length:", np.mean(lens))
    print("Maximun review length:", np.max(lens))
    print("Minimun review length:", np.min(lens))
    print("Standard deviation of review length:", np.std(lens))

#### Tokenize reviews

In [55]:
tk_reviews = [tokenizeReview(r) for r in reviews] 

In [57]:
print("Raw review:\n",reviews[emp],'\n')
print("Tokenized review:\n",tk_reviews[emp])

Raw review:
 b'while it was part of former yugoslavia , my country , croatia , used to be almost completely ignored by western films in general . \nwords " croatia " and " croatians " were unheard , even in movies whose plots actually took place on croatian locations . \nthe rare exception was the dogs of war , 1980 adaptation of frederick forsyth\'s bestselling novel , yet that rare occasion only added insult to the injury , by mentioning croatians in most unflattering context . \nin a single line they are referred as a bunch of mean people , probably terrorists , who happen to remove parts of arm dealers\' stomachs when they are unsatisfied with the quality of merchandise . \nsame line also illustrates the milieu where the movie takes place - dark and violent world of shady international deals with the blurred boundaries between business , politics and crime . \nthe movie protagonist , shannon ( christopher walken ) is a mercenary who spent the best years of his life fighting numerou

In [59]:
stats_print(tk_reviews)

Vocabulary size:  48639
Total number of tokens:  1291842
Lexical diversity:  0.037650889195427924
Total number of reviews: 2002
Average review length: 645.2757242757243
Maximun review length: 2366
Minimun review length: 16
Standard deviation of review length: 286.01108817236855


### 2.2. Removing Single Character Token


In [66]:
st_list = [[w for w in review if len(w) <= 1 ] for review in tk_reviews] # create a list of single character token for each review
list(chain.from_iterable(st_list)) # merge them together in one list

['a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'i',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'a',
 'i',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'i',
 'r',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 's',
 'a',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 's',
 'i',
 'a',
 'a',
 'i',
 'a',
 'a',
 'i',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 's',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 's',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a'

In [68]:
# filter out single character tokens
tk_reviews = [[w for w in review if len(w) >=2] for review in tk_reviews]

In [74]:
# check whether the single character token has been removed
print("Tokenized review:\n",tk_reviews[emp])

Tokenized review:
 ['while', 'it', 'was', 'part', 'of', 'former', 'yugoslavia', 'my', 'country', 'croatia', 'used', 'to', 'be', 'almost', 'completely', 'ignored', 'by', 'western', 'films', 'in', 'general', 'words', 'croatia', 'and', 'croatians', 'were', 'unheard', 'even', 'in', 'movies', 'whose', 'plots', 'actually', 'took', 'place', 'on', 'croatian', 'locations', 'the', 'rare', 'exception', 'was', 'the', 'dogs', 'of', 'war', 'adaptation', 'of', 'frederick', "forsyth's", 'bestselling', 'novel', 'yet', 'that', 'rare', 'occasion', 'only', 'added', 'insult', 'to', 'the', 'injury', 'by', 'mentioning', 'croatians', 'in', 'most', 'unflattering', 'context', 'in', 'single', 'line', 'they', 'are', 'referred', 'as', 'bunch', 'of', 'mean', 'people', 'probably', 'terrorists', 'who', 'happen', 'to', 'remove', 'parts', 'of', 'arm', "dealers'", 'stomachs', 'when', 'they', 'are', 'unsatisfied', 'with', 'the', 'quality', 'of', 'merchandise', 'same', 'line', 'also', 'illustrates', 'the', 'milieu', 'wher

### 2.3. Removing Stop words
Get English `stopwords` from `ntlk`

In [85]:
stopwords = stopwords.words('english')

Ignore **negative** stopwords

In [93]:
ignored_words = [w for w in stopwords if not ("not" in w or "n't" in w or "no" in w)]

In [95]:
# filter out stop words
tk_reviews = [[w for w in review if w not in ignored_words] for review in tk_reviews]

In [99]:
# examine the reviews stat
stats_print(tk_reviews)

Vocabulary size:  48468
Total number of tokens:  713889
Lexical diversity:  0.06789290772094822
Total number of reviews: 2002
Average review length: 356.5879120879121
Maximun review length: 1392
Minimun review length: 8
Standard deviation of review length: 154.5078277786966


### 2.4. Stemming and Lemmatization
Use`WordNet` for lemmatization 

In [114]:
words = list(chain.from_iterable(tk_reviews)) 
vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words

In [117]:
lemmatizer = WordNetLemmatizer()

# examine the lemmatizer first
ch_list = ['{0} -> {1}'.format(w, lemmatizer.lemmatize(w)) for w in vocab if w!=lemmatizer.lemmatize(w)]
print(len(ch_list))
ch_list

5132


['carvings -> carving',
 'sugarplums -> sugarplum',
 'stonecutters -> stonecutter',
 'suspects -> suspect',
 'conditions -> condition',
 'parkas -> parka',
 'shafts -> shaft',
 'boosts -> boost',
 'voyeurs -> voyeur',
 'traces -> trace',
 'maids -> maid',
 'memoirs -> memoir',
 'programs -> program',
 'rentals -> rental',
 'smokers -> smoker',
 'teachers -> teacher',
 'chases -> chase',
 'insiders -> insider',
 'professions -> profession',
 'remnants -> remnant',
 'serves -> serf',
 'davies -> davy',
 'molecules -> molecule',
 'dazzles -> dazzle',
 'hinterlands -> hinterland',
 'vessels -> vessel',
 'extremes -> extreme',
 'dials -> dial',
 'finesses -> finesse',
 'scanners -> scanner',
 'uplifts -> uplift',
 'transfusions -> transfusion',
 'lice -> louse',
 'maps -> map',
 'tots -> tot',
 'streets -> street',
 'violins -> violin',
 'issues -> issue',
 'promises -> promise',
 'eyes -> eye',
 'lovers -> lover',
 'strings -> string',
 'kicks -> kick',
 'pronouns -> pronoun',
 'complement

In [119]:
# lemmatize each review
tk_reviews = [[lemmatizer.lemmatize(w) for w in review] for review in tk_reviews]

In [121]:
stats_print(tk_reviews)

Vocabulary size:  44130
Total number of tokens:  713889
Lexical diversity:  0.06181633279123225
Total number of reviews: 2002
Average review length: 356.5879120879121
Maximun review length: 1392
Minimun review length: 8
Standard deviation of review length: 154.5078277786966
